# Short PyTorch Introduction

Wir gehen hier kurz auf die wichtigsten Konzepte von PyTorch ein, da wir es in der nächsten Übung verwenden werden. In dieser kurzen Introduction schauenwir uns kurz ein paar der wichtigsten Konzepte in PyTorch an und trainieren ein einfaches neuronales Netzwerk. Tiefergehende Informationen findet ihr in der PyTorch Dokumentation zum Beispiel [hier](https://pytorch.org/tutorials/beginner/basics/intro.html).

### Imports

In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt

### Tensoren

Tensoren sind das Fundament in PyTorch und vergleichbar mit Numpy-Arrays. Im Gegensatz zu Numpy-Arrays haben Tensoren eine `.device` Eigenschaft, die angibt, auf welchem Gerät (CPU oder GPU) sie sich befinden.

In [ ]:
data = [[1, 2],[3, 4]]
test_tensor = torch.tensor(data)
test_tensor

In [ ]:
print('Test Tensor on device: ', test_tensor.device)

Wir können mit dem folgenden Code prüfen, ob eine GPU verfügbar ist, und verschieben die Daten ggf. auf die GPU (Wenn ihr Google colab verwendet, könnt ihr eine Runtime mit GPU auswählen): 

In [ ]:
print('GPU available: ', torch.cuda.is_available())

if torch.cuda.is_available():
    test_tensor = test_tensor.to("cuda")

print('Test Tensor on device: ', test_tensor.device)

### Daten laden
Wir laden und verarbeiten hier wie in der Übung den MNIST-Datensatz, der handgeschriebene Ziffern enthält. Die Daten werden für das Training vorbereitet.

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Lambda(lambda x: x.squeeze(0))  # Remove the single channel dimension
])

# Load the MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transform, download=True)

In [ ]:
train_dataset[0]

In [ ]:
# Display image and label.
features, label = train_dataset[0]
plt.imshow(features, cmap="gray")
plt.show()
print(f"Label: {label}")

### Batch-Verarbeitung mit DataLoader
Der Datensatz gibt uns für ein gegebenen index die Features und Label von genau einem sample zurück. Beim Trainieren eines Modells wollen wir mehrere Samples in „Minibatches“ an das Modell übergeben, die Daten bei jeder Epoche neu mischen, um eine Overfitting des Modells zu vermeiden, und das Multiprocessing von Python nutzen, um den Datenabruf zu beschleunigen.

DataLoader abstrahiert die diese Komplexität für uns in einer einfachen API:

In [ ]:
batch_size = 128

# Create DataLoaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

### Neuronales Netz definieren
Wir definieren unser neuronales Netz als `nn.Module` und initialisieren die Struktur des neuronalen Netzes in `__init__`. Jede `nn.Module`-Unterklasse implementiert die Operationen mit den Eingabedaten in der Forward-Methode.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, structure):
        super().__init__()
        self.flatten = nn.Flatten()
        layers = []

        for i in range(len(structure) - 1):
            layers.append(nn.Linear(structure[i], structure[i + 1]))
            if i < len(structure) - 2:  # Add activation for all layers except the last
                layers.append(nn.Sigmoid())
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.flatten(x)
        logits = self.model(x)
        return logits


In [ ]:
structure = [784, 128, 10]
learning_rate = 0.001 # Testet das Modell z.B. auch mal mit einer größeren Learning Rate, z.B. 0.1. Wie ändert sich die Performance?
epochs = 10

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = NeuralNetwork(structure).to(device)

### Trainings- und Testschleife

Wir trainieren das Modell und evaluieren es in jeder Epoche. Die Trainingsschleife berechnet die Verluste und aktualisiert die Gewichte. Die Backpropagation findet in den folgenden drei Zeilen statt:
- `loss.backward()` : Berechnet die Gradienten der Verlustfunktion bezüglich der Gewichte
- `optimizer.step()` : Aktualisiert die Gewichte basierend auf den Gradienten und der Lernrate
- `optimizer.zero_grad()` : Setzt die Gradienten, die in den `.grad`-Attributen der Parameter gespeichert sind, wieder auf Null zurück für den nächsten Durchlauf

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    ### Train loop

    size = len(train_loader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(train_loader):
        # Move data and labels to the same device as the model
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    ### Test loop

    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in test_loader:
            # Move data and labels to the same device as the model
            X, y = X.to(device), y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

print("Done!")

### Backpropagation mit `torch.autograd` verstehen

Um diese Gradienten zu berechnen, hat PyTorch eine integrierte Differentiation Engine namens `torch.autograd`. 
Wir betrachten um das zu veranschaulichen hier das einfachste einschichtige neuronale Netz mit der Eingabe `x`, den Parametern `w` und `b` und einer Lossfunktion. 
In diesem Netz sind `w` und `b` Parameter, die wir optimieren müssen. Daher müssen wir in der Lage sein, die Gradienten der Verlustfunktion in Bezug auf diese Variablen zu berechnen. Aus diesem Grund setzen wir die Eigenschaft `requires_grad` dieser Tensoren auf `True`.

In [ ]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

![](comp-graph.png)

Für Funktionen, die wir in PyTorch auf Tensoren anwenden, um den Computational Graph zu konstruieren,  ist unterliegend nicht nur definiert wie die Vorwärtsrichtung zu berechnen ist, sondern auch wie ihre Ableitung während des Backpropagationsschritts zu berechnen ist. Ein Verweis auf die zugehörige Backpropagationsfunktion in dem Computational Graph wird in der Eigenschaft `grad_fn` eines Tensors gespeichert:

In [ ]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Um die Gewichte der Parameter im neuronalen Netz zu optimieren, müssen wir die Ableitungen unserer Verlustfunktion in Bezug auf die Parameter berechnen, d. h. wir benötigen 
$ \frac{\partial L}{\partial w} $ und $ \frac{\partial L}{\partial b} $.
Um diese Ableitungen über unsern computational graph zu berechnen, rufen wir `loss.backward()` auf.
Anschließend könenn wir dien Gradienten für die einzelnen Parameter in `w.grad` und `b.grad` finden.
Bis zu diesem Punkt haben wir nur die Gradienten berechnet, aber noch keine Gewichtsaktualisierung durchgeführt. 
Um die Gewichte zu aktualisieren, müssten wir zuvor einen Optimizer initialisieren, der die Gewichte aktualisiert.
Dann könnten wir wie zuvor in dem Trainingloop `optimizer.step()` aufrufen, um die Gewichte basierend auf den Gradienten und der Lernrate zu aktualisieren.

In [ ]:
loss.backward()
print(w.grad)
print(b.grad)

Pytorch's Autograd hält die Daten (Tensoren) und alle durchgeführten Operationen (zusammen mit den daraus resultierenden neuen Tensoren) in einem gerichteten azyklischen Graphen (DAG) fest, der aus Funktionsobjekten besteht. In diesem DAG sind die Blätter die Eingabe-Tensoren und die Wurzeln die Ausgabe-Tensoren. Wenn man diesen Graphen von den Wurzeln zu den Blättern zurückverfolgt, kann man die Gradienten unter Verwendung der Kettenregel automatisch berechnen.

In einem Forward Pass führt autograd zwei Dinge gleichzeitig aus:
- die angeforderte Operation durchführen, um einen resultierenden Tensor zu berechnen
- Beibehaltung der Gradientenfunktion der Operation in der DAG.

Der Backward Pass beginnt, wenn .backward() auf der DAG-Wurzel (loss) aufgerufen wird. Autograd
- berechnet die Gradienten von jedem .grad_fn,
- akkumuliert sie im Attribut .grad eines jeden Tensors bis zum Input